#  Training on Multiple GPUs with `gluon`

In this notebook, we will
- introduce how MXNet handles parallelism
- implement data parallel training for LeNet (GPUs required).

In [1]:
# check the number of GPUs available
!nvidia-smi

Mon Aug 20 20:39:24 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.81                 Driver Version: 384.81                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   40C    P0    24W / 300W |      0MiB / 16152MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   40C    P0    24W / 300W |      0MiB / 16152MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## How Does MXNet Handle Parallelism?

Wiriting parallel code in Python is non-trivial. What about in MXNet?


In MXNet, operations are

- executed asynchronously, and

- scheduled according to dependency

### Asynchronous Execution

- Operations are pushed to the backend engine and executed asynchronously.

- The python code is blocked only when `print(y)` or `y.asnumpy()` is called and the result is not ready.

Workloads, such as nd.dot are pushed into the backend engine for lazy evaluation. That is, Python merely pushes the workload nd.dot and returns immediately without waiting for the computation to be finished. We keep pushing until the results need to be copied out from MXNet, such as print(x) or are converted into numpy by x.asnumpy(). At that time, the Python thread is blocked until the results are ready.

In [1]:
import mxnet as mx
from mxnet import nd, gluon, autograd
from time import time
import utils

In [2]:
start = time()
i = nd.random.uniform(shape=(2000,2000))
j = nd.dot(i, i.T)
print('Operations are pushed into the backend engine\n%f s' % (time() - start))
j.asnumpy()
print('Operations are done and the result is ready\n%f s' % (time() - start))

Operations are pushed into the backend engine
0.002873 s
Operations are done and the result is ready
0.217274 s


### Dependency Scheduling

Independent operations may be scheduled to run in parallel by MXNet.

MXNet depends on a powerful scheduling algorithm that analyzes the dependencies of the pushed workloads. This scheduler checks to see if two workloads are independent of each other. If they are, then the engine may run them in parallel. If a workload depend on results that have not yet been computed, it will be made to wait until its inputs are ready.

For example, if we call three operators:
```
a = nd.random_uniform(...)
b = nd.random_uniform(...)
c = a + b
```
Then the computation for a and b may run in parallel, while c cannot be computed until both a and b are ready.


#### Defining NDArrays and Operations

In [3]:
# create NDArrays on GPU
x0 = nd.random.uniform(shape=(4000, 4000), ctx=mx.gpu(0))
x1 = nd.random.uniform(shape=(4000, 4000), ctx=mx.gpu(1))

def run(x):
    """push 10 matrix-matrix multiplications"""
    return [nd.dot(x,x) for i in range(10)]

def wait(x):
    """explicitly wait until all results are ready"""
    for y in x:
        y.wait_to_read()

#### Comparing Sequential and Parallel Runs

In [4]:
print('Run on GPU 0 and 1 in sequential')
start = time()
wait(run(x0))
wait(run(x1))
print('time: %f sec' %(time() - start))
print('Run on GPU 0 and 1 in parallel')
start = time()
y0 = run(x0)
y1 = run(x1)
wait(y0)
wait(y1)
print('time: %f sec' %(time() - start))

Run on GPU 0 and 1 in sequential
time: 0.245280 sec
Run on GPU 0 and 1 in parallel
time: 0.106554 sec


## Data Parallelism for Deep Learning

For deep learning, data parallelism is by far the most widely used approach for partitioning workloads. It works like this: Assume that we have k GPUs. We split the examples in a data batch into k parts, and send each part to a different GPUs which then computes the gradient that part of the batch. Finally, we collect the gradients from each of the GPUs and sum them together before updating the weights.

<img src="data_parallel.png" width='800px'>

## Training ResNet34 V2 with Multiple GPUs


Define the neural network and loss function.

In [5]:
net = gluon.model_zoo.vision.resnet34_v2(classes=10)
loss = gluon.loss.SoftmaxCrossEntropyLoss()

### Initializing on Multiple Devices

Gluon supports initialization of network parameters over multiple devices. We accomplish this by passing in an array of device contexts, instead of the single contexts we've used in earlier notebooks.
When we pass in an array of contexts, the parameters are initialized 
to be identical across all of our devices.

In [6]:
ctx = [mx.gpu(0), mx.gpu(1)]
num_devices = len(ctx)
net.hybridize(static_alloc=True, static_shape=True)
net.initialize(ctx=ctx)

### Loading MNIST 

Given a batch of input data,
we can split it into parts (equal to the number of contexts) 
by calling `gluon.utils.split_and_load(batch, ctx)`.
The `split_and_load` function doesn't just split the data,
it also loads each part onto the appropriate device context. 

So now when we call the forward pass on two separate parts,
each one is computed on the appropriate corresponding device and using the version of the parameters stored there.

In [7]:
mnist_train = utils.normalize_and_copy(gluon.data.vision.MNIST(train=True), ctx=ctx[1])
mnist_test = utils.normalize_and_copy(gluon.data.vision.MNIST(train=False), ctx=ctx[1])
batch_size = 64
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

### Split Data for Forward and Backward Computation

In [8]:
i, (data, label) = next(enumerate(train_data))
data_list = gluon.utils.split_and_load(data[:4], ctx)
label_list = gluon.utils.split_and_load(label[:4], ctx)
print(net(data_list[0]))
print(net(data_list[1]))


[[-0.18367606  0.38539067  0.0810388  -0.36636704  0.24176018 -0.12539324
   0.0618905   0.21979626  0.3648352   0.27388057]
 [-0.1568249   0.560107    0.20823881 -0.58401716  0.5127848   0.00183654
   0.21796837  0.22614041  0.4221663   0.36251718]]
<NDArray 2x10 @gpu(0)>

[[-0.09060284  0.3859282   0.3972354  -0.7385969   0.34928346  0.01967236
  -0.02816124  0.43405226  0.3601955   0.20923229]
 [-0.07137774  0.23342773  0.13774091 -0.4162132   0.35673463  0.01140356
   0.09675159  0.1791319   0.22318979  0.21127512]]
<NDArray 2x10 @gpu(1)>


At any time, we can access the version of the parameters stored on each device. 
Recall from the first Chapter that our weights may not actually be initialized
when we call `initialize` because the parameter shapes may not yet be known. 
In these cases, initialization is deferred pending shape inference. 

#### Inspect Weights of the First Layer on Different Devices

In [9]:
weight = net.features[1].weight
for c in ctx:
    weight_on_ctx = weight.data(ctx=c)
    print('=== channel 0 of the first conv on {} ==={}'.format(
        c, weight_on_ctx[0]))

=== channel 0 of the first conv on gpu(0) ===
[[[ 0.03582338 -0.05703442  0.02046952  0.00537236  0.01164322
   -0.01281808  0.03641203]
  [-0.03176162 -0.060988    0.06300376 -0.03806128 -0.02654165
   -0.00634275  0.02375414]
  [ 0.00347787  0.06492888  0.00712219  0.0277181   0.00173751
    0.0262342  -0.01722588]
  [ 0.03967453 -0.06445411  0.02099127  0.05131169 -0.01271829
   -0.02123801 -0.03724146]
  [ 0.01555783  0.0639345   0.01833989  0.02040232  0.02875151
   -0.06902379  0.01204208]
  [-0.01736174  0.02835646 -0.06423311  0.05263435  0.05001806
    0.06768566 -0.03909312]
  [ 0.03614714  0.005475    0.03496752 -0.041912    0.05475691
    0.01937268  0.00913627]]]
<NDArray 1x7x7 @gpu(0)>
=== channel 0 of the first conv on gpu(1) ===
[[[ 0.03582338 -0.05703442  0.02046952  0.00537236  0.01164322
   -0.01281808  0.03641203]
  [-0.03176162 -0.060988    0.06300376 -0.03806128 -0.02654165
   -0.00634275  0.02375414]
  [ 0.00347787  0.06492888  0.00712219  0.0277181   0.00173751


#### Inspect Gradients of the First Layer on Different Devices

In [10]:
def forward_backward(net, data_list, label_list):
    with autograd.record():
        losses = [loss(net(X), Y) for X, Y in zip(data_list, label_list)]
    for l in losses:
        l.backward()

Similarly, we can access the gradients on each of the GPUs. Because each GPU gets a different part of the batch (a different subset of examples), the gradients on each GPU vary. 

In [11]:
forward_backward(net, data_list, label_list)
for c in ctx:
    grad_on_ctx = weight.grad(ctx=c)
    print('=== grad of channel 0 of the first conv2d on {} ==={}'.format(
        c, grad_on_ctx[0]))

=== grad of channel 0 of the first conv2d on gpu(0) ===
[[[  3517.4688 -44901.09   -89587.13   -66705.73    20949.812
    53379.555   57688.137 ]
  [ 31140.346  -16102.486  -44485.695  -43412.918   26441.758
    73986.58    31593.041 ]
  [ 49982.047   10538.945  -12221.834  -32302.422   50121.273
    66072.21    21097.793 ]
  [ 47142.953    7937.446   -9989.615   -3465.6895  -2173.338
    40656.98     5405.1484]
  [ 75860.35    38860.39   -11847.77   -20935.316  -10120.942
    30186.033    5465.3755]
  [ 52975.566   16579.865   -8577.469  -33620.63   -21547.572
    -9651.623   -9236.727 ]
  [ 43617.996   50427.5      6384.574  -36300.83   -35787.734
   -42527.055  -36163.684 ]]]
<NDArray 1x7x7 @gpu(0)>
=== grad of channel 0 of the first conv2d on gpu(1) ===
[[[ 2.9299689e+04  1.8483438e+04  8.5118613e+03  6.6304614e+02
    3.5992820e+04  2.3011988e+04  1.6259367e+04]
  [ 1.8721021e+04  2.3937648e+04 -2.4922852e+01 -1.7493465e+04
    1.7566496e+04 -1.2260168e+03 -2.1855119e+04]
  [-5.19

## Put all things together

Now we can implement the remaining functions. 

#### Define Train and Validation Function

In [12]:
def train_batch(data, label, ctx, net, trainer):
    # split the data batch and load them on GPUs
    data_list = gluon.utils.split_and_load(data, ctx)
    label_list = gluon.utils.split_and_load(label, ctx)
    # compute gradient
    forward_backward(net, data_list, label_list)
    # update parameters
    trainer.step(data.shape[0])

def valid_batch(data, label, ctx, net):
    data = data.as_in_context(ctx[0])
    pred = nd.argmax(net(data), axis=1, keepdims=True)
    return nd.sum(pred == label.as_in_context(pred.context)).asscalar()

In [13]:
def run(ctx, batch_size, lr):    
    # data iterator
    train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
    valid_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)
    print('Batch size is {}'.format(batch_size))
    net.collect_params().initialize(force_reinit=True, ctx=ctx)
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
    for epoch in range(2):
        # train
        start = time()
        for data, label in train_data:
            train_batch(data, label, ctx, net, trainer)
        mx.nd.waitall()
        print('Epoch %d, training time = %.1f sec'%(epoch, time()-start))
        # validating
        correct, num = 0.0, 0.0
        for data, label in valid_data:
            correct += valid_batch(data, label, ctx, net)
            num += data.shape[0]
        print('         validation accuracy = %.4f'%(correct/num))

In [14]:
# single GPU
ctx_list = [ctx[0]]
print('Running on {}'.format(ctx_list))
run(ctx_list, 128*len(ctx_list), .04)
# multi-GPU
ctx_list = ctx
print('Running on {}'.format(ctx_list))
run(ctx_list, 128*len(ctx_list), .08)

Running on [gpu(0)]
Batch size is 128
Epoch 0, training time = 10.3 sec
         validation accuracy = 0.9879
Epoch 1, training time = 9.5 sec
         validation accuracy = 0.9859
Running on [gpu(0), gpu(1)]
Batch size is 256
Epoch 0, training time = 8.5 sec
         validation accuracy = 0.9498
Epoch 1, training time = 7.6 sec
         validation accuracy = 0.9841


## Conclusion

Gluon makes it easy to implement data parallel training.

Both parameters and trainers in `gluon` support multi-devices. Moving from one device to multi-devices is straightforward. 